# 4.10 Coding Etiquette & Excel Reporting

## Part 1

### 1. Import the data set you exported in Exercise 4.9 into a new Jupyter notebook. This should be the merged version of your data that contains the new customers column. 

In [1]:
import pandas as pd 
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
path = r'D:\02.2022_Instacart Basket Analysis'

In [3]:
df_cop_merged= pd.read_pickle(os.path.join(path,'02 Data','Prepared Data','customer_orders_production_merge.pkl'))

### 2. Consider any security implications that might exist for this new data. You’ll need to address any PII data in the data before continuing your analysis.

In [ ]:
df_cop_merged.head()

In [5]:
df_cop_merged_PIIchecked = df_cop_merged.drop(columns = ['first_name','last_name'])

Names are personally identifiable information, so I dropped these columns here. Moreover,I would like to check all other columns whether they also contain PII, so I cancelled display_max limitation as below.

In [6]:
pd.set_option('display.max_columns', False)

In [7]:
df_cop_merged_PIIchecked.head()

,user_id,gender,state,age,n_dependants,family_status,income,order_id,order_number,orders_day_of_week,order_time_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,spending,spending_flag,frequency,frequency_flag
0,26711,Female,Missouri,48,3,married,165665,2543867,5,1,9,30.0,False,196,2,0,Soda,77,7,9.0,Regularly busy,Busiest days,Average orders,8,New customer,7.99,Low spender,19.0,Regular customer
1,26711,Female,Missouri,48,3,married,165665,1285508,7,5,15,11.0,False,196,1,1,Soda,77,7,9.0,Regularly busy,Regularly busy,Most orders,8,New customer,7.99,Low spender,19.0,Regular customer
2,26711,Female,Missouri,48,3,married,165665,2578584,8,1,15,10.0,False,196,2,1,Soda,77,7,9.0,Regularly busy,Busiest days,Most orders,8,New customer,7.99,Low spender,19.0,Regular customer
3,26711,Female,Missouri,48,3,married,165665,423547,2,2,9,14.0,False,38928,1,0,0% Greek Strained Yogurt,120,16,12.6,Regularly busy,Regularly busy,Average orders,8,New customer,7.99,Low spender,19.0,Regular customer
4,26711,Female,Missouri,48,3,married,165665,2524893,3,3,11,30.0,False,38928,1,1,0% Greek Strained Yogurt,120,16,12.6,Regularly busy,Least busy days,Most orders,8,New customer,7.99,Low spender,19.0,Regular customer


No PII information in the other columns. 

### 3. The Instacart officers are interested in comparing customer behavior in different geographic areas. Create a regional segmentation of the data. 
### (1.) You’ll need to create a “Region” column based on the “State” column from your customers data set. Use the region information in this Wikipedia article to create your column (you only need to create regions, not  divisions).
### (2.) Determine whether there’s a difference in spending habits between the different U.S. regions. (Hint: You can do this by crossing the variable you just created with the spending flag.)

In [8]:
# Check if there is misspelling in state column
df_cop_merged_PIIchecked['state'].unique()

array(['Missouri', 'New Mexico', 'Idaho', 'Iowa', 'Maryland', 'Kentucky',
       'Montana', 'South Carolina', 'Texas', 'Virginia', 'Nevada',
       'Nebraska', 'Georgia', 'Wyoming', 'Colorado', 'North Dakota',
       'Wisconsin', 'Alaska', 'Vermont', 'Arkansas', 'Maine',
       'North Carolina', 'West Virginia', 'Indiana', 'Oregon', 'Florida',
       'California', 'Pennsylvania', 'Ohio', 'Connecticut', 'Arizona',
       'Louisiana', 'Washington', 'New York', 'Mississippi', 'Oklahoma',
       'Utah', 'New Hampshire', 'Hawaii', 'District of Columbia',
       'Alabama', 'Massachusetts', 'Rhode Island', 'Michigan',
       'New Jersey', 'Kansas', 'South Dakota', 'Minnesota', 'Illinois',
       'Tennessee', 'Delaware'], dtype=object)

No misspelling is found.

In [9]:
# Create Region list
Northeast = ['Maine','New Hampshire','Vermont','Massachusetts','Rhode Island','Connecticut','New York','New Jersey','Pennsylvania']
Midwest = ['Wisconsin','Michigan','Illinois','Indiana','Ohio','North Dakota','South Dakota','Nebraska','Kansas','Minnesota','Iowa','Missouri']
South = ['Delaware','Maryland','District of Columbia','Virginia','West Virginia','North Carolina','South Carolina','Georgia','Florida','Kentucky','Tennessee','Mississippi','Alabama','Oklahoma','Texas','Arkansas','Louisiana']
West = ['Idaho','Montana','Wyoming','Nevada','Utah','Colorado','Arizona','New Mexico','Alaska','Washington','Oregon','California','Hawaii']

In [10]:
# Assign regions according to each state
df_cop_merged_PIIchecked.loc[df_cop_merged_PIIchecked['state'].isin(['Maine','New Hampshire','Vermont','Massachusetts','Rhode Island','Connecticut','New York','New Jersey','Pennsylvania']),'region'] = 'Northeast'
df_cop_merged_PIIchecked.loc[df_cop_merged_PIIchecked['state'].isin(['Wisconsin','Michigan','Illinois','Indiana','Ohio','North Dakota','South Dakota','Nebraska','Kansas','Minnesota','Iowa','Missouri']),'region'] = 'Midwest'
df_cop_merged_PIIchecked.loc[df_cop_merged_PIIchecked['state'].isin(['Delaware','Maryland','District of Columbia','Virginia','West Virginia','North Carolina','South Carolina','Georgia','Florida','Kentucky','Tennessee','Mississippi','Alabama','Oklahoma','Texas','Arkansas','Louisiana']),'region'] = 'South'
df_cop_merged_PIIchecked.loc[df_cop_merged_PIIchecked['state'].isin(['Idaho','Montana','Wyoming','Nevada','Utah','Colorado','Arizona','New Mexico','Alaska','Washington','Oregon','California','Hawaii']),'region'] = 'West'

In [11]:
# Check if there is still NaN in region column
df_cop_merged_PIIchecked['region'].value_counts(dropna =False)

South        10197331
West          7830688
Midwest       7175716
Northeast     5421220
Name: region, dtype: int64

In [ ]:
# Another method:
# using np.select add regions column
# cond=[sales_cust['state'].isin(Northeast),sales_cust['state'].isin(Midwest),sales_cust['state'].isin(South),sales_cust['state'].isin(West)]
# values=['Northeast','Midwest','South','West']
# sales_cust['region']=np.select(cond,values)

### 4. The Instacart CFO isn’t interested in customers who don’t generate much revenue for the app. Create an exclusion flag for low-activity customers (customers with less than 5 orders) and exclude them from the data. Make sure you export this sample.

In [12]:
df_cop_merged_PIIchecked.columns

Index(['user_id', 'gender', 'state', 'age', 'n_dependants', 'family_status',
       'income', 'order_id', 'order_number', 'orders_day_of_week',
       'order_time_of_day', 'days_since_prior_order', 'first_order',
       'product_id', 'add_to_cart_order', 'reordered', 'product_name',
       'aisle_id', 'department_id', 'prices', 'busiest_day', 'busiest_days',
       'busiest_period_of_day', 'max_order', 'loyalty_flag', 'spending',
       'spending_flag', 'frequency', 'frequency_flag', 'region'],
      dtype='object')

In [13]:
# Create activity flags by loc functions
df_cop_merged_PIIchecked.loc[df_cop_merged_PIIchecked['max_order'] < 5,'activity_flag'] = 'low_activity'

In [14]:
df_cop_merged_PIIchecked.loc[df_cop_merged_PIIchecked['max_order'] >= 5,'activity_flag'] = 'normal_activity'

In [15]:
# check the result of loc functions
df_cop_merged_PIIchecked['activity_flag'].value_counts(dropna = False)

normal_activity    29266394
low_activity        1358561
Name: activity_flag, dtype: int64

In [16]:
df_cop_merged_PIIchecked.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30624955 entries, 0 to 30624954
Data columns (total 31 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   user_id                 int32  
 1   gender                  object 
 2   state                   object 
 3   age                     int8   
 4   n_dependants            int8   
 5   family_status           object 
 6   income                  int32  
 7   order_id                int32  
 8   order_number            int8   
 9   orders_day_of_week      int8   
 10  order_time_of_day       int8   
 11  days_since_prior_order  float64
 12  first_order             bool   
 13  product_id              int32  
 14  add_to_cart_order       int16  
 15  reordered               int8   
 16  product_name            object 
 17  aisle_id                int8   
 18  department_id           int8   
 19  prices                  float64
 20  busiest_day             object 
 21  busiest_days            objec

In [ ]:
# Check values in columns to see if we can change datatype to save memory space

In [17]:
df_cop_merged_PIIchecked['days_since_prior_order'].max()

30.0

In [18]:
df_cop_merged_PIIchecked['prices'].max()

25.0

In [19]:
df_cop_merged_PIIchecked['spending'].max()

23.2

In [20]:
df_cop_merged_PIIchecked['frequency'].max()

30.0

In [ ]:
# Change datatype to save memory space

In [21]:
df_cop_merged_PIIchecked['days_since_prior_order'] = df_cop_merged_PIIchecked['days_since_prior_order'].astype('float16')

In [22]:
df_cop_merged_PIIchecked['prices'] = df_cop_merged_PIIchecked['prices'].astype('float16')

In [23]:
df_cop_merged_PIIchecked['spending'] = df_cop_merged_PIIchecked['spending'].astype('float16')

In [24]:
df_cop_merged_PIIchecked['frequency'] = df_cop_merged_PIIchecked['frequency'].astype('float16')

In [25]:
df_cop_merged_PIIchecked['max_order'] = df_cop_merged_PIIchecked['max_order'].astype('int8')

In [ ]:
# Filter out low-active customers

In [27]:
df_cop_merged_PIIchecked = df_cop_merged_PIIchecked.loc[df_cop_merged_PIIchecked['activity_flag'] =='normal_activity']

In [30]:
df_cop_merged_PIIchecked.head()

,user_id,gender,state,age,n_dependants,family_status,income,order_id,order_number,orders_day_of_week,order_time_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,spending,spending_flag,frequency,frequency_flag,region,activity_flag
0,26711,Female,Missouri,48,3,married,165665,2543867,5,1,9,30.0,False,196,2,0,Soda,77,7,9.000000,Regularly busy,Busiest days,Average orders,8,New customer,7.988281,Low spender,19.0,Regular customer,Midwest,normal_activity
1,26711,Female,Missouri,48,3,married,165665,1285508,7,5,15,11.0,False,196,1,1,Soda,77,7,9.000000,Regularly busy,Regularly busy,Most orders,8,New customer,7.988281,Low spender,19.0,Regular customer,Midwest,normal_activity
2,26711,Female,Missouri,48,3,married,165665,2578584,8,1,15,10.0,False,196,2,1,Soda,77,7,9.000000,Regularly busy,Busiest days,Most orders,8,New customer,7.988281,Low spender,19.0,Regular customer,Midwest,normal_activity
3,26711,Female,Missouri,48,3,married,165665,423547,2,2,9,14.0,False,38928,1,0,0% Greek Strained Yogurt,120,16,12.601562,Regularly busy,Regularly busy,Average orders,8,New customer,7.988281,Low spender,19.0,Regular customer,Midwest,normal_activity
4,26711,Female,Missouri,48,3,married,165665,2524893,3,3,11,30.0,False,38928,1,1,0% Greek Strained Yogurt,120,16,12.601562,Regularly busy,Least busy days,Most orders,8,New customer,7.988281,Low spender,19.0,Regular customer,Midwest,normal_activity


In [31]:
df_cop_merged_PIIchecked.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29266394 entries, 0 to 30624913
Data columns (total 31 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   user_id                 int32  
 1   gender                  object 
 2   state                   object 
 3   age                     int8   
 4   n_dependants            int8   
 5   family_status           object 
 6   income                  int32  
 7   order_id                int32  
 8   order_number            int8   
 9   orders_day_of_week      int8   
 10  order_time_of_day       int8   
 11  days_since_prior_order  float16
 12  first_order             bool   
 13  product_id              int32  
 14  add_to_cart_order       int16  
 15  reordered               int8   
 16  product_name            object 
 17  aisle_id                int8   
 18  department_id           int8   
 19  prices                  float16
 20  busiest_day             object 
 21  busiest_days            objec

In [33]:
# Exporting
df_cop_merged_PIIchecked.to_pickle(os.path.join(path,'02 Data','Prepared Data','customer_orders_production_merge.pkl'))